In [1]:
library(GenomicRanges)
library(data.table)  # 高效读取大文件
library(S4Vectors)
library(BSgenome.Hsapiens.UCSC.hg38)
library(JASPAR2020)
library(TFBSTools)
library(motifmatchr)
library(ArchR)
library(ggplot2)
library(chromVAR)
library(Seurat)
library(future.apply)

Warning message:
“程序包‘GenomicRanges’是用R版本4.4.2 来建造的”
载入需要的程序包：stats4

载入需要的程序包：BiocGenerics

Warning message:
“程序包‘BiocGenerics’是用R版本4.4.2 来建造的”

载入程序包：‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, saveRDS, setdiff,
    table, tapply, union, unique, unsplit, which.max, which.min


载入需要的程序包：S4Vectors

Warning message:
“程序包‘S4Vectors’是用R版本4.4.2 来建造的”

载入程序包：‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


载入需要的程序包：IRanges

载入需要的程序包：GenomeInfoDb

In [2]:
packageVersion("TFBSTools")

[1] ‘1.44.0’

In [4]:
differentialDeviations <- function(object, 
                                  groups,
                                  alternative = c("two.sided", "less", "greater"), 
                                  parametric = TRUE) {
  stopifnot(is(object, "chromVARDeviations"))
  
  # 解析 groups
  if (length(groups) == 1 && groups %in% colnames(colData(object))) {
    groups <- colData(object)[[groups]]
  } else if (length(groups) != ncol(object)) {
    stop("invalid groups input, must be vector of length ncol(object) or column name from colData(object)")
  }
  
  groups <- as.factor(groups)
  alternative <- match.arg(alternative)
  inputs <- deviations(object)  # motifs x cells

  # 存储结果
  p_val <- rep(NA_real_, nrow(inputs))
  fc <- rep(NA_real_, nrow(inputs))  # 存储 log2 fold change
  direction <- rep(NA_character_, nrow(inputs))

  if (parametric) {
    if (nlevels(groups) == 2) {
      # 两组 t-test + FC
      res <- apply(inputs, 1, function(x) {
        grp_means <- tapply(x, groups, mean)
        names(grp_means) <- make.names(names(grp_means))  # 防止空格等问题
        diff_mean <- diff(grp_means)  # group2 - group1直接用均值相减去
        fc <- grp_means[2] / (grp_means[1] + 1e-9)  # 防止除零
        #l2fc <- log2(fc)
        
        test <- t.test(x ~ groups, alternative = alternative)
        
        dir <- ifelse(fc > 0, 
                      paste0(levels(groups)[2], " > ", levels(groups)[1]),
                      paste0(levels(groups)[1], " > ", levels(groups)[2]))
        
        return(list(
          p.value = test$p.value,
          fc = fc,
          direction = dir
        ))
      })
      p_val <- sapply(res, `[[`, "p.value")
      fc <- sapply(res, `[[`, "fc")
      direction <- sapply(res, `[[`, "direction")
      
    } else {
      # 多组 ANOVA：返回最大组与其余组均值的 fc
      p_val <- apply(inputs, 1, function(x) {
        oneway.test(x ~ groups)$p.value
      })
      
      fc <- apply(inputs, 1, function(x) {
        grp_means <- tapply(x, groups, mean)
        max_group <- names(which.max(grp_means))
        mean_others <- mean(grp_means[names(grp_means) != max_group])
        fc <- grp_means[max_group] / (mean_others + 1e-9)
        #log2(fc)
      })
      
      direction <- apply(inputs, 1, function(x) {
        grp_means <- tapply(x, groups, mean)
        max_group <- names(which.max(grp_means))
        paste0("max in: ", max_group)
      })
    }
  } else {
    if (nlevels(groups) == 2) {
      # 非参数：Wilcoxon + median difference
      res <- apply(inputs, 1, function(x) {
        medians <- tapply(x, groups, median)
        fc <- medians[2] / (medians[1] + 1e-9)
        #l2fc <- log2(fc)
        
        test <- wilcox.test(x ~ groups, alternative = alternative)
        
        dir <- ifelse(fc > 0,
                      paste0(levels(groups)[2], " > ", levels(groups)[1]),
                      paste0(levels(groups)[1], " > ", levels(groups)[2]))
        
        return(list(
          p.value = test$p.value,
          fc = fc,
          direction = dir
        ))
      })
      p_val <- sapply(res, `[[`, "p.value")
      fc <- sapply(res, `[[`, "fc")
      direction <- sapply(res, `[[`, "direction")
      
    } else {
      # Kruskal-Wallis + median-based fc
      p_val <- apply(inputs, 1, function(x) {
        kruskal.test(x ~ groups)$p.value
      })
      
      fc <- apply(inputs, 1, function(x) {
        medians <- tapply(x, groups, median)
        max_group <- names(which.max(medians))
        med_others <- mean(medians[names(medians) != max_group])
        fc <- medians[max_group] / (med_others + 1e-9)
        #log2(fc)
      })
      
      direction <- apply(inputs, 1, function(x) {
        medians <- tapply(x, groups, median)
        max_group <- names(which.max(medians))
        paste0("max median in: ", max_group)
      })
    }
  }
  
  # 多重检验校正
  p_adj <- p.adjust(p_val, method = "BH")
  
  # 返回结果
  return(data.frame(
    p_value = p_val,
    p_value_adj = p_adj,
    fold_change = fc,
    direction = direction,
    row.names = rownames(inputs)
  ))
}

In [6]:
input_bed_dir <- "/media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/motif_chromVAR/Input_bed"
seurat_dir    <- "/media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/seurat"
output_root   <- "/media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/motif_chromVAR/output/"

z_matrix_dir        <- file.path(output_root, "z_matrix")
deviations_matrix_dir <- file.path(output_root, "deviations_matrix")
diff_dir            <- file.path(output_root, "diff")

# 创建输出目录
dir.create(z_matrix_dir, recursive = TRUE, showWarnings = FALSE)
dir.create(deviations_matrix_dir, recursive = TRUE, showWarnings = FALSE)
dir.create(diff_dir, recursive = TRUE, showWarnings = FALSE)

bed_path <- file.path(input_bed_dir, "L4_bed.txt")  # 固定 BED 文件

# --- 1. 获取所有 Seurat 对象对应的细胞类型 ---
seurat_files <- list.files(seurat_dir, pattern = "\\.rds$", full.names = FALSE)
all_celltypes <- gsub("\\.rds$", "", seurat_files)

if (length(all_celltypes) == 0) {
  stop(" 在目录中未找到任何 .rds 文件: ", seurat_dir)
}

# --- 2. 获取 deviations_matrix 目录中已存在的细胞类型 ---
# 假设每个细胞类型的输出文件是：<celltype>_deviation_matrix.rds
existing_files <- list.files(deviations_matrix_dir, pattern = "_dev\\.rds$", full.names = FALSE)
existing_celltypes <- gsub("_dev\\.rds$", "", existing_files)

# --- 3. 计算需要处理的细胞类型（差集）
celltypes <- setdiff(all_celltypes, existing_celltypes)

if (length(celltypes) == 0) {
  message("所有细胞类型均已处理完毕，无需重新计算。")
} else {
  message("发现 ", length(celltypes), " 个待处理的细胞类型: ", paste(celltypes, collapse = ", "))
}

# --- 4. 并行处理函数（无需修改，future 会自动导出环境）---
process_celltype <- function(celltype) {
  message("开始处理细胞类型: ", celltype)
  
  seurat_path <- file.path(seurat_dir, paste0(celltype, ".rds"))
  
  if (!file.exists(bed_path)) stop("BED 文件不存在: ", bed_path)
  if (!file.exists(seurat_path)) stop(" Seurat 对象不存在: ", seurat_path)
  
  tryCatch({
    # 加载数据
    peaks <- getPeaks(bed_path, sort_peaks = TRUE)
    atac <- readRDS(seurat_path)
    meta <- atac@meta.data
    
    if (!"BMI_group" %in% colnames(meta)) {
      stop("meta.data 中缺少 'BMI_group' 列")
    }
    
    counts_matrix <- GetAssayData(atac, slot = "counts")
    fragment_counts <- SummarizedExperiment(
      assays = list(counts = counts_matrix),
      rowRanges = peaks
    )
    
    fragment_counts <- addGCBias(fragment_counts, genome = BSgenome.Hsapiens.UCSC.hg38)
    fragment_counts <- filterPeaks(fragment_counts, min_fragments_per_peak = 1, non_overlapping = TRUE)
    
    matrices <- getMatrixSet(
      JASPAR2020,
      opts = list(species = 9606, all = FALSE, factorType = "PFM")
    )
    
    motif_ix <- matchMotifs(matrices, fragment_counts, genome = BSgenome.Hsapiens.UCSC.hg38)
    bg <- getBackgroundPeaks(object = fragment_counts)
    dev <- computeDeviations(object = fragment_counts, annotations = motif_ix, background_peaks = bg)
    
    variability <- computeVariability(dev)
    colData(dev) <- cbind(colData(dev), meta)
    
    z_matrix <- assay(dev, "z")
    groups <- meta$BMI_group
    target_groups <- c("normal", "Underweight", "Overweight")
    
    keep_motif <- apply(z_matrix, 1, function(devs) {
      non_na_count <- tapply(!is.na(devs), groups, sum)
      all(non_na_count[target_groups] > 0, na.rm = TRUE)
    })
    
    dev_filtered <- dev[keep_motif, ]
    message("[", celltype, "] 筛选后保留 motif 数: ", nrow(dev_filtered))
    
    # # --- 差异偏差分析 ---
    # results_list_dev <- list()
    # for (pair in combn(target_groups, 2, simplify = FALSE)) {
    #   grp1 <- pair[1]; grp2 <- pair[2]
    #   keep_cells <- groups %in% c(grp1, grp2)
    #   dev_sub <- dev_filtered[, keep_cells]
    #   new_groups <- factor(groups[keep_cells], levels = c(grp1, grp2))
      
    #   res <- differentialDeviations(dev_sub, groups = new_groups, parametric = TRUE)
    #   res$motif <- rownames(res)
    #   res$comparison <- paste(grp1, "vs", grp2)
    #   res$group1 <- grp1
    #   res$group2 <- grp2
    #   rownames(res) <- NULL
    #   results_list_dev[[length(results_list_dev) + 1]] <- res
    # }
    
    # diff_acc_pairwise <- do.call(rbind, results_list_dev)
    # diff_acc_pairwise$name <- variability[diff_acc_pairwise$motif, "name"]
    # diff_acc_pairwise_sig <- diff_acc_pairwise[diff_acc_pairwise$p_value_adj < 0.05, ]
    
    # if (nrow(diff_acc_pairwise_sig) == 0) {
    #   message(" [", celltype, "] 差异偏差分析无显著结果 (p_value_adj < 0.05)")
    # } else {
    #   write.csv(diff_acc_pairwise_sig, file.path(diff_dir, paste0(celltype, "_diff_acc_sig.csv")))
    #   message("[", celltype, "] 已保存 ", nrow(diff_acc_pairwise_sig), " 个显著差异偏差结果")
    # }
    
    # # --- 差异变异性分析 ---
    # results_list_var <- list()
    # for (pair in combn(target_groups, 2, simplify = FALSE)) {
    #   grp1 <- pair[1]; grp2 <- pair[2]
    #   keep_cells <- groups %in% c(grp1, grp2)
    #   dev_sub <- dev_filtered[, keep_cells]
    #   new_groups <- factor(groups[keep_cells], levels = c(grp1, grp2))
      
    #   res <- differentialVariability(dev_sub, groups = new_groups, parametric = TRUE)
    #   inputs <- deviationScores(dev_sub)#The deviationScores are the Z-scores for each bias corrected deviations. 
    #   high_var_group <- apply(inputs, 1, function(x) {
    #     vars <- tapply(x, new_groups, var, na.rm = TRUE)
    #     names(vars)[which.max(vars)]
    #   })
      
    #   res$high_var_group <- high_var_group
    #   res$motif <- rownames(res)
    #   res$comparison <- paste(grp1, "vs", grp2)
    #   res$group1 <- grp1
    #   res$group2 <- grp2
    #   rownames(res) <- NULL
    #   results_list_var[[length(results_list_var) + 1]] <- res
    # }
    # diff_var <- do.call(rbind, results_list_var)
    # diff_var_sig <- diff_var[diff_var$p_value_adj < 0.05, ]
    # if (nrow(diff_var_sig) == 0) {
    #   message(" [", celltype, "] 差异变异性分析无显著结果 (p_value_adj < 0.05)")
    # } else {
    #   write.csv(diff_var_sig, file.path(diff_dir, paste0(celltype, "_diff_var_sig.csv")))
    #   message("[", celltype, "] 已保存 ", nrow(diff_var_sig), " 个显著差异变异性结果")
    # }
    
    # --- 保存矩阵 ---
    saveRDS(assay(dev, "z"), file.path(z_matrix_dir, paste0(celltype, "_z.rds")))
    saveRDS(assay(dev, "deviations"), file.path(deviations_matrix_dir, paste0(celltype, "_dev.rds")))
    
    return(list(celltype = celltype, success = TRUE))
    
  }, error = function(e) {
    message("[", celltype, "] 处理失败: ", e$message)
    return(list(celltype = celltype, success = FALSE, error = e$message))
  })
}

# --- 5. 设置并行后端（自动适配平台）---
num_cores <- 4

# 使用 multisession（Windows 安全）或 multicore（Unix）
plan(multisession, workers = num_cores)  # Windows/Linux/macOS 都支持

message("启动并行计算，使用 ", num_cores, " 个工作进程...")

# --- 6. 并行执行 ---
results <- future_lapply(celltypes, process_celltype)

# --- 7. 汇总结果 ---
failed <- results[sapply(results, function(r) !r$success)]
if (length(failed) == 0) {
  message("所有 ", length(celltypes), " 个细胞类型处理成功！")
} else {
  message(" 以下细胞类型处理失败:")
  for (f in failed) {
    message("    - ", f$celltype, " : ", f$error)
  }
}

message("批量分析完成！")

🔍 发现 64 个待处理的细胞类型: AS_DC, Atypical_Bm_ITGAX, Bn_IFIT3, Bn_TCL1A, CD4_CTL_GZMH, CD4_Tcm_CXCR5, CD4_Tcm_IFI44L, CD4_Tem_CCR5, CD4_Tem_CCR7neg, CD4_Tfh-like_CXCR5, CD4_Th_CCR4, CD4_Th_LMNA, CD4_Th_TNFRSF11A, CD4_Th17-like_GZMK, CD4_Th17-like_RORC, CD4_Th22-like_CCR10, CD4_Tn_CCR7, CD4_Tr1_IL10, CD4_Treg_FCRL3, CD4_Treg_FOXP3, CD8_CTL_GZMB, CD8_CTL_GZMK, CD8_CTL_IFI44L, CD8_Tcm_IFI44L, CD8_Tem_CCR7neg, CD8_Tn_CCR7, cMono_CD14, cMono_CXCL10, cMono_IFI44L, cMono_IL1B, DC_CSF2RA, DC1_CLEC9A, DC2_CD1C, gdT1_TRDV1, gdT2_GZMH, gdT2_GZMK, gdT2_IL12RB2, HSPC_CD34, ILC2_IL2RA, intMono_GFRA2, MAIT_SLC4A10, Mature_NK_dim_FCGR3A, MK_GP9, ncMono_C1QA, ncMono_FCGR3A, ncMono_IFIT1, NK_bright_XCL1, NKT_IFNG, NKT_NCR1, pDC_IRF4, Plasma_IGHA1, Plasma_IGHG1, Plasmablast_MKI67, pre-Switched_Bm_JAM3, Proliferative_NK_MKI67, Proliferative_T_MKI67, Switched_activated_Bm_CD86, Switched_Bm_IGHDneg, Switched_Bm_IGHE, Terminal_NK_dim_CD160neg, Transitional_B_MME, Transitional_NK_GZMK, Unswitched_Bm_CD1C, Unswitched_

In [5]:
    matrices <- getMatrixSet(
      JASPAR2020,
      opts = list(species = 9606, all = TRUE, factorType = "PFM")
    )

In [6]:
matrices

PFMatrixList of length 3141
names(3141): MA0001.1 MA0002.1 MA0003.1 MA0004.1 ... MA1704.1 UN0383.1 UN0384.1